# Chapter 49: Policy Gradient Methods

### This code generates the numerical results for Examples 5, 6, 8 and 9  in Chapter 48: Value Function Approximation (vol. II)
TEXT: A. H. Sayed, INFERENCE AND LEARNING FROM DATA, Cambridge University Press, 2022.

<div style="text-align: justify">
DISCLAIMER:  This computer code is  provided  "as is"   without  any  guarantees.
Practitioners  should  use it  at their own risk.  While  the  codes in  the text 
are useful for instructional purposes, they are not intended to serve as examples 
of full-blown or optimized designs. The author has made no attempt at optimizing 
the codes, perfecting them, or even checking them for absolute accuracy. In order 
to keep the codes at a level  that is  easy to follow by students, the author has 
often chosen to  sacrifice  performance or even programming elegance in  lieu  of 
simplicity. Students can use the computer codes to run variations of the examples 
shown in the text. 
</div>

The Jupyter notebook and python codes are developed by Eduardo Faria Cabrera

required libraries:
    
1. numpy
2. matplotlib
3. tqdm

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from functions import *

## Example 49.5 (Playing a game over a grid)

We illustrate the operation of the advantage actor--critic (A2C) algorithm (49.77)   by reconsidering the same grid problem from  Fig. 48.1. We assign one-hot encoded feature vectors with the actions $a\in\mathbb{A}$ and collect them into a matrix of size $5\times 5$ (one row per action):

$$
A=\begin{bmatrix}1&0&0&0&0\\0&1&0&0&0\\0&0&1&0&0\\0&0&0&1&0\\0&0&0&0&1\end{bmatrix}
\begin{array}{l}\leftarrow \textnormal{ up}\\
\leftarrow \textnormal{ down}\\
\leftarrow \textnormal{ left}\\
\leftarrow \textnormal{ right}\\
\leftarrow \textnormal{ stop}
\end{array} \tag{49.79}
$$

We continue to employ the same one-hot encoded feature vectors (48.14) from Example 48.1, which we collected into a $17\times 17$ matrix $H$ with one row corresponding to each state:

$$
H=\begin{array}{|ccccccccccccccccc|}
1 &0 &0 &0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0 &1 &0 &0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0 &0 &1 &0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0 &0 &0 &1&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0 &0 &0 &0&1&0&0&0&0&0&0&0&0&0&0&0&0\\
0 &0 &0 &0&0&1&0&0&0&0&0&0&0&0&0&0&0\\
0 &0 &0 &0&0&0&1&0&0&0&0&0&0&0&0&0&0\\
0 &0 &0 &0&0&0&0&1&0&0&0&0&0&0&0&0&0\\
0 &0 &0 &0&0&0&0&0&1&0&0&0&0&0&0&0&0\\
0 &0 &0 &0&0&0&0&0&0&1&0&0&0&0&0&0&0\\
0 &0 &0 &0&0&0&0&0&0&0&1&0&0&0&0&0&0\\
0 &0 &0 &0&0&0&0&0&0&0&0&1&0&0&0&0&0\\
0 &0 &0 &0&0&0&0&0&0&0&0&0&1&0&0&0&0\\
0 &0 &0 &0&0&0&0&0&0&0&0&0&0&1&0&0&0\\
0 &0 &0 &0&0&0&0&0&0&0&0&0&0&0&1&0&0\\
0 &0 &0 &0&0&0&0&0&0&0&0&0&0&0&0&1&0\\
0 &0 &0 &0&0&0&0&0&0&0&0&0&0&0&0&0&0
\end{array}\begin{array}{l}\leftarrow 1\\\leftarrow2\\\leftarrow3\\\leftarrow4\\\leftarrow5\\\leftarrow6\\\leftarrow7\\\leftarrow8\\\leftarrow9\\\leftarrow10\\\leftarrow11\\\leftarrow12\\\leftarrow13\\\leftarrow14\\\leftarrow15\\\leftarrow16\\\leftarrow17\end{array} \tag{49.80}
$$

Using the matrices $H$ and $A$, we define feature vectors $f_{s,a}$ for the state--action pairs $(s,a)$ by computing the Kronecker product of $H$ and $A$ as follows:

$$
F \overset{\Delta}{=} H\otimes A \tag{49.81}
$$

The resulting matrix $F$ has dimensions $85\times 85$, with each row in $F$ corresponding to the transpose of the feature vector $f_{s,a}$ associated with the state--action pair $(s,a)$. For any particular $(s,a)$, the row in $F$ that corresponds to it has index: 

$$
\textnormal{ row index}\;=\;\Bigl\{(s-1)\times |\mathbb{A}|\;+\;a\Bigr\},\;\;\;\begin{array}{l}
s=1,2,\ldots, |\mathbb{S}|\\
a=1,2,\ldots, |\mathbb{A}|
\end{array} \tag{49.82}
$$

where $ |\mathbb{S}|=17$ and $ |\mathbb{A}|=5$. We run the A2C algorithm for 1,000,000 iterations using constant step sizes:

$$
\mu_1=0.01,\;\; \mu=0.0001, \;\;\rho=0.001,\;\;\gamma=0.9 \tag{49.83}
$$

Each iteration involves a run over $E=100$ episodes to train the critic. At the end of the simulation, we evaluate the policies at each state and arrive at the  matrix $\Pi$ in (49.84), with each row corresponding to one state and each column to one action (i.e., each row corresponds to the distribution $\pi(a|s)$). 
The boxes highlight the largest entries in each column and are used to define the "optimal" policies at that state.  The resulting optimal actions are represented by arrows in Fig. 49.2. 

$$
\Pi=\begin{array}{ccccc|l}
\textnormal{ up}&\textnormal{ down}&\textnormal{ left}&\textnormal{ right}&\textnormal{ stop}&\textnormal{ state}\\\hline
0.0020  &  \fbox{0.8653}  &  0.1253   & 0.0074     &    0&\leftarrow s=1\\
\fbox{0.9254}  &  0.0144 &   0.0540 &   0.0061  &       0 &\leftarrow s=2\\
\fbox{0.9156}  &  0.0250  &  0.0337  &  0.0257    &     0&\leftarrow s=3\\
0&0&0&0&0&\leftarrow s=4\\
\fbox{0.9735} &   0.0080 &   0.0128  &  0.0057   &      0&\leftarrow s=5\\
0.0230 &   0.0094  & \fbox{ 0.9429}  &  0.0247   &      0&\leftarrow s=6\\
\fbox{0.9840} &   0.0026 &   0.0124   & 0.0011    &     0&\leftarrow s=7\\
0&0&0&0&\fbox{1.0000}&\leftarrow s=8\\
\fbox{0.9940} &   0.0008   & 0.0024   & 0.0027  &       0&\leftarrow s=9\\
\fbox{0.9905}   & 0.0013  &  0.0032  &  0.0050   &      0&\leftarrow s=10\\
0&0&0&0&0&\leftarrow s=11\\
\fbox{0.9878}  &  0.0025 &   0.0049  &  0.0049     &    0&\leftarrow s=12\\
0.0058  &  0.0027   & 0.0037   & \fbox{0.9878}     &    0&\leftarrow s=13\\
0.0030 &   0.0030  &  0.0016  &  \fbox{0.9924} &        0&\leftarrow s=14\\
 0.0019  &  0.0010   & 0.0009  &  \fbox{0.9963}   &      0&\leftarrow s=15\\
0&0&0&0&\fbox{1.0000}&\leftarrow s=16\\
0&0&0&0&\fbox{1.0000}&\leftarrow s=17\end{array} \tag{49.84}
$$

In [2]:
# grid problem from example 1

# states
# we include the block locations 4 and 11 for convenience of coding; though they will never be reached
states = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17] # s = 17 is the EXIT state
NS = len(states) # number of states

# actions
actions = ['up', 'down', 'left', 'right', 'stop']
NA = len(actions) # number of actions

# rewards
reward = -0.1*np.ones(NS)
reward[7] = -10 # reward at state s = 8
reward[15] = +10 # reward at state s = 16
reward[16] = 0 # reward at exit satate s = 17

# target policy pi(a|s)
Pi = np.zeros((NA, NS)) # matrix Pi specifies the policy pi(a|s)
                      # each row is an action; each column is a state

for j in range(NS):
    s = states[j]
    if s in [1, 2, 3, 5, 6, 7, 9, 10, 12, 13, 14, 15]:
        Pi[0,j] = 1/4 # up
        Pi[1,j] = 1/4 # down
        Pi[2,j] = 1/4 # left
        Pi[3,j] = 1/4 # right
        Pi[4,j] = 0  # STOP
    
    else:
        Pi[0,j] = 0 # up
        Pi[1,j] = 0 # down
        Pi[2,j] = 0 # left
        Pi[3,j] = 0 # right
        Pi[4,j] = 1 # STOP

# transition kernel
P = np.zeros((NS, NA, NS)) # entries are Prob(s, a, s')

P[0, 0, 0] = 0.15 # start at s=1, move UP, end in state 1
P[0, 0, 1] = 0.15
P[0, 0, 7] = 0.7

P[0, 1, 0] = 0.85 # start at s=1, move DOWN, end in state 1
P[0, 1, 1] = 0.15

P[0,2,0] = 0.15 # start at s=1, move LEFT, end in state 1
P[0,2,1] = 0.70
P[0,2,7] = 0.15

P[0,3,0] = 0.85 # start at s=1, move RIGHT, end in state 1
P[0,3,7] = 0.15

P[1,0,0] = 0.15  # start at s=2, move UP, end in state 1
P[1,0,2] = 0.15
P[1,0,6] = 0.70

P[1,1,0] = 0.15  # start at s=2, move DOWN, end in state 1
P[1,1,2] = 0.15
P[1,1,1] = 0.70
                
P[1,2,2] = 0.70  # start at s=2, move LEFT, end in state 3                  
P[1,2,6] = 0.15
P[1,2,1] = 0.15

P[1,3,0] = 0.70   # start at s=2, move RIGHT, end in state 1 
P[1,3,6] = 0.15
P[1,3,1] = 0.15

P[2,0,5] = 0.70   # start at s=3, move UP
P[2,0,2] = 0.15
P[2,0,1] = 0.15

P[2,1,2] = 0.85  # start at s=3, move DOWN
P[2,1,1] = 0.15

P[2,2,2] = 0.85  # start at s=3, move LEFT
P[2,2,5] = 0.15 

P[2,3,1] = 0.70   # start at s=3, move RIGHT
P[2,3,5] = 0.15
P[2,3,4] = 0.15

P[4,0,11] = 0.70  # start at s=5, move UP
P[4,0,4]  = 0.15
P[4,0,5]  = 0.15

P[3,0,3] = 1 # values for location 4 this state is never reached
P[3,1,3] = 1 # so these values are irrelevant
P[3,2,3] = 1
P[3,3,3] = 1
P[3,4,3] = 1

P[4,1,4] = 0.85  # start at s=5, move DOWN
P[4,1,5] = 0.15

P[4,2,4]  = 0.85  # start at s=5, move LEFT
P[4,2,11] = 0.15

P[4,3,5]  = 0.70  # start at s=5, move RIGHT
P[4,3,11] = 0.15
P[4,3,4]  = 0.15

P[5,0,4] = 0.15   # start at s=6, move UP
P[5,0,5] = 0.70
P[5,0,6] = 0.15

P[5,1,2] = 0.70    # start at s=6, move DOWN
P[5,1,4] = 0.15
P[5,1,6] = 0.15

P[5,2,4] = 0.70   # start at s=6, move LEFT
P[5,2,5] = 0.15
P[5,2,2] = 0.15

P[5,3,6] = 0.70   # start at s=6, move RIGHT
P[5,3,5] = 0.15
P[5,3,2] = 0.15

P[6,0,9] = 0.70  # start at s=7, move UP
P[6,0,5]  = 0.15
P[6,0,7]  = 0.15

P[6,1,1] = 0.70  # start at s=7, move DOWN
P[6,1,5] = 0.15
P[6,1,7] = 0.15

P[6,2,5]  = 0.70 # start at s=7, move LEFT
P[6,2,9] = 0.15
P[6,2,1]  = 0.15

P[6,3,7] = 0.70  # start at s=7, move RIGHT
P[6,3,1] = 0.15
P[6,3,9] = 0.15

P[7,0,16] = 0   # start at s=8 [DANGER] EXIT
P[7,1,16] = 0
P[7,2,16] = 0
P[7,3,16] = 0
P[7,4,16] = 1 #STOP action

P[8,0,15] = 0.70   # start at s=9 move UP
P[8,0,9] = 0.15
P[8,0,8]  = 0.15

P[8,1,7]  = 0.70   # start at s=9 move DOWN
P[8,1,9] = 0.15
P[8,1,8]  = 0.15

P[8,2,9] = 0.70  # start at s=9 move LEFT
P[8,2,15] = 0.15
P[8,2,7]  = 0.15

P[8,3,8]  = 0.70  # start at s=9 move RIGHT
P[8,3,7]  = 0.15
P[8,3,15] = 0.15

P[9,0,14] = 0.70   # start at s=10 move UP
P[9,0,8]  = 0.15
P[9,0,9] = 0.15

P[9,1,6]  = 0.70  # start at s=10 move DOWN
P[9,1,8]  = 0.15
P[9,1,9] = 0.15

P[9,2,9] = 0.70  # start at s=10 move LEFT
P[9,2,14] = 0.15
P[9,2,6]  = 0.15

P[9,3,8]  = 0.70   # start at s=10 move RIGHT
P[9,3,6]  = 0.15
P[9,3,14] = 0.15

P[10,0,3] = 1 # values for location 11 this state is never reached
P[10,1,3] = 1 # so these values are irrelevant
P[10,2,3] = 1
P[10,3,3] = 1
P[10,4,3] = 1

P[11,0,12] = 0.70  # start at s=12 move UP
P[11,0,11] = 0.30

P[11,1,4]  = 0.70  # start at s=12 move DOWN
P[11,1,11] = 0.30

P[11,2,12] = 0.15  # start at s=12 move LEFT
P[11,2,4]  = 0.15
P[11,2,11] = 0.70

P[11,3,11] = 0.70  # start at s=12 move RIGHT
P[11,3,4]  = 0.15
P[11,3,12] = 0.15

P[12,0,12] = 0.85 # start at s=13 move UP
P[12,0,13] = 0.15

P[12,1,11] = 0.70  # start at s=13 move DOWN
P[12,1,12] = 0.15
P[12,1,13] = 0.15

P[12,2,12] = 0.85 # start at s=13 move LEFT
P[12,2,11] = 0.15

P[12,3,13] = 0.70  # start at s=13 move RIGHT
P[12,3,11] = 0.15
P[12,3,12] = 0.15

P[13,0,13] = 0.70 # start at s=14 move UP
P[13,0,12] = 0.15
P[13,0,14] = 0.15

P[13,1,13] = 0.70  # start at s=14 move DOWN
P[13,1,12] = 0.15
P[13,1,14] = 0.15

P[13,2,12] = 0.70  # start at s=14 move LEFT
P[13,2,13] = 0.30

P[13,3,14] = 0.70  # start at s=14 move RIGHT
P[13,3,13] = 0.30

P[14,0,14] = 0.70  # start at s=15 move UP
P[14,0,13] = 0.15
P[14,0,15] = 0.15

P[14,1,9] = 0.70   # start at s=15 move DOWN
P[14,1,13] = 0.15
P[14,1,15] = 0.15

P[14,2,13] = 0.70  # start at s=15 move LEFT
P[14,2,9] = 0.15
P[14,2,14] = 0.15

P[14,3,15] = 0.70   # start at s=15 move RIGHT
P[14,3,9] = 0.15
P[14,3,14] = 0.15

P[15,0,16] =0   # start at s=16 [REWARD] EXIT
P[15,1,16] =0
P[15,2,16] =0
P[15,3,16] =0
P[15,4,16] =1 # STOP action

P[16,0,16] = 0
P[16,1,16] = 0
P[16,2,16] = 0
P[16,3,16] = 0
P[16,4,16] = 1 # EXIT state

# Computing rpi(s)
rpi = np.zeros(NS)
for s in range(NS):
    policy = Pi[:, s]
    for a in range(NA):
        for sprime in range(NS):
            rpi[s] += policy[a]*P[s, a, sprime]*reward[s]

# Computing P^{\pi}
Ppi = np.zeros((NS, NS))
for s in range(NS):
    policy = Pi[:, s]
    for sprime in range(NS):
        for a in range(NA):
            Ppi[s, sprime] += policy[a]*P[s, a, sprime]

# behavior policy phi(a|s) used to simulate off-policy algorithms
Phi = np.zeros((NA, NS)) # matri Phi specifies the behavior policy phi(a|s)
                         # each row is an action; each column is a state

for j in range(NS):
    s = states[j]
    if s in [1, 2, 3, 5, 6, 7, 9, 10, 12, 13, 14, 15]:
        Phi[0,j] = 3/8 # up
        Phi[1,j] = 1/8 # down
        Phi[2,j] = 2/6 # left
        Phi[3,j] = 1/6 # right
        Phi[4,j] = 0  # STOP
    else:
        Phi[0,j] = 0 # up
        Phi[1,j] = 0 # down
        Phi[2,j] = 0 # left
        Phi[3,j] = 0 # right
        Phi[4,j] = 1  # STOP

# one-hot encoding for the actions
A = np.zeros((5, 5))
A[0, :] = np.array([1, 0, 0, 0, 0]) # up
A[1, :] = np.array([0, 1, 0, 0, 0]) # down
A[2, :] = np.array([0, 0, 1, 0, 0]) # left
A[3, :] = np.array([0, 0, 0, 1, 0]) # right
A[4, :] = np.array([0, 0, 0, 0, 1]) # STOP

# 4x1 reduced feature vectors with four binary entries
# is agent on same row as SUCCESS
# is agent on same row as DANGER
# is agent in rightmost two columns
# is agent in leftmost two columns

# reduced features for state-value function
# no offset is included in the feature vectors since v^{\pi}=0 at state 17
# v^{\pi}(s) = h'*w

Mr = 4
Hr = np.zeros((NS, Mr))
Hr[0,:]  = np.array([0, 0, 1, 0]) # state 1
Hr[1,:]  = np.array([0, 0, 1, 0]) # state 2
Hr[2,:]  = np.array([0, 0, 0, 1]) # state 3
Hr[3,:]  = np.array([0, 0, 0, 0]) # not a valid state
Hr[4,:]  = np.array([0, 1, 0, 1]) # state 5...
Hr[5,:]  = np.array([0, 1, 0, 1])
Hr[6,:]  = np.array([0, 1, 1, 0])
Hr[7,:]  = np.array([0, 1, 1, 0])
Hr[8,:]  = np.array([0, 0, 1, 0])
Hr[9,:] = np.array([0, 0, 1, 0])
Hr[10,:] = np.array([0, 0, 0, 0]) # not a valid state 
Hr[11,:] = np.array([0, 0, 0, 1])
Hr[12,:] = np.array([1, 0, 0, 1])
Hr[13,:] = np.array([1, 0, 0, 1])
Hr[14,:] = np.array([1, 0, 1, 0])
Hr[15,:] = np.array([1, 0, 1, 0]) # state 16
Hr[16,:] = np.array([0, 0, 0, 0]) # EXIT state

Fr = np.kron(Hr, A) # Kronecker product of dimensions (NSxNA) x (MrxNA)
Tr = Mr*NA

# one-hot encoded feature vectors for state-value function
# no offset is included in the feature vectors because v^{\pi}=0 at state 17
# v^{\pi}(s) = h'*w

Me = NS
He = np.zeros((NS, Me))
He[0,:]   = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) # state 1
He[1,:]   = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) # state 2
He[2,:]   = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) # state 3
He[3,:]   = np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) # not valid state
He[4,:]   = np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) # state 5
He[5,:]   = np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) # ...
He[6,:]   = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
He[7,:]   = np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
He[8,:]   = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
He[9,:]  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
He[10,:]  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]) # not valid state
He[11,:]  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
He[12,:]  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
He[13,:]  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
He[14,:]  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
He[15,:]  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]) # state 16
He[16,:]  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) # EXIT state

Fe = np.kron(He, A) # Kronecker product of dimensions (NSxNA) x (MexNA)
Te = Me*NA


In [4]:
# Advantage actor-critic (A2C) algorithm for optimal policy design

E = 100 # number of episodes within each iteration
max_episode_duration = 50
delta = 0
gamma = 0.9
mu = 0.01 # step-size for critic (w)
mug = 0.001 # step-size for actor (theta)
iter_ = 30000 # repeat for this many iterations to learn theta; each iteration has E episodes
use_reduced_features = 0 # set to zero to use the one-hot encoded extended feature

if use_reduced_features == 1:
    H = Hr 
    M = Mr 
    F = Fr 
    T = Tr 
else:
    H = He 
    M = Me 
    F = Fe 
    T = Te 

w = np.zeros(M) # linear value function model; includes bias coefficient
theta = np.zeros(T) # parameter for Gibbs distribution

saved_states = [] # need to save states, features, rewards actions during all episodes
saved_features = []
saved_rewards = []
saved_actions = []

kernel = np.zeros(NS)

for m in tqdm(range(iter_)): # each iteration involves multiple episodes
    # Current Gibbs distribution using current model theta
    Pi_theta = compute_policy(F,theta,T,NA,NS,Pi)
    counter_transitions = 1 # counts # of state transitions during this iteration

    for e in range(E): # iterates over episodes
        counter = 0
        sample = 1
        while sample == 1:
            idx = np.random.randint(NS-1)+1 # select a random non-exit state index
            if (idx != 4) and (idx != 11) and (idx != 17): # excluding the block locations and exit state
                s = states[idx-1]
                sample = 0

        while (s!=17) and (counter < max_episode_duration): # state s different from EXIT state
            h = H[s, :].T # state feature vector 
            policy = Pi_theta[:, s] # pi(a|s; theta) at state s
            act = select_action(policy)

            for j in range(NS):
                kernel[j] = P[s, act, j]
            
            sprime = select_next_state(kernel) # next state
            hprime = H[sprime, :].T # next feature

            # CRITIC for state value function learning
            r = reward[s] # in this example reward is only state s-dependent
            delta = r + (gamma*hprime-h).T@w
            w += mu*delta*h 

            #saved visited states, actions, rewards
            saved_features.append([h, hprime])
            saved_states.append([s, sprime])
            saved_rewards.append(r)
            saved_actions.append(act)

            s = sprime # set next sample
            counter += 1
            counter_transitions += 1 # plays role of N
    wo = w.copy() # learned CRITIC MODEL

    # ACTOR
    N = counter_transitions - 1
    sum_g = np.zeros(T)

    for nn in range(N):
        hx = saved_features[nn][0]
        hxprime = saved_features[nn][1]
        sx = saved_states[nn][0]
        sxprime = saved_states[nn][1]
        rx = saved_rewards[nn]
        ax = saved_actions[nn]

        deltax = rx + (gamma*hxprime - hx).T@wo 
        g_vec = compute_gradient_log_pi(F,Pi_theta,sx,ax,NA) # gradient of log pi
        sum_g += deltax*g_vec
    sum_g = sum_g/N 
    theta += mug*sum_g

100%|██████████| 30000/30000 [2:35:45<00:00,  3.21it/s]     


In [7]:
thetao = theta 
Pi_theta = compute_policy(F,thetao,T,NA,NS,Pi) # optimal policy
max_values = Pi_theta.max(axis=0)
indexes = np.argmax(Pi_theta, axis=0)

act = [0]*NS
action_states = [0]*NS
for s in range(NS):
    act[s] = indexes[s] # indexes of the permissible action  
    action_states[s] = actions[act[s]]
    print(s+1, action_states[s])

print("policy AxS from ADVANTAGE (A2C) actor-critic algorithm")
print(Pi_theta)

1 up
2 left
3 up
4 stop
5 up
6 left
7 up
8 stop
9 up
10 up
11 stop
12 up
13 right
14 right
15 right
16 stop
17 stop
policy AxS from ADVANTAGE (A2C) actor-critic algorithm
[[0.25       0.24975992 0.25921322 0.         0.26622445 0.25720326
  0.33403165 0.         0.31233695 0.30668719 0.         0.27029995
  0.25375439 0.24507499 0.26901846 0.         0.        ]
 [0.25       0.24902915 0.24818634 0.         0.2518691  0.24656925
  0.22680526 0.         0.1750736  0.18271397 0.         0.22957249
  0.22975958 0.23821982 0.22405228 0.         0.        ]
 [0.25       0.28095767 0.25624102 0.         0.25546835 0.27963903
  0.30373133 0.         0.23153432 0.25495129 0.         0.25105832
  0.23354345 0.20869731 0.203011   0.         0.        ]
 [0.25       0.22025327 0.23635943 0.         0.22643811 0.21658846
  0.13543176 0.         0.28105512 0.25564755 0.         0.24906925
  0.28294258 0.30800787 0.30391826 0.         0.        ]
 [0.         0.         0.         1.         0.     

## Example 49.6 (Playing a game over a grid)

We illustrate the operation of the natural-gradient actor--critic algorithm (49.91)   by reconsidering the same grid problem from  Fig. 48.1. We employ the same encoding for states $s$ and state--action pairs $(s,a)$ from Example 49.5.  We run the algorithm for 10,000 iterations using constant step sizes

$$
\mu_1=0.01,\;\; \mu=0.0001, \;\;\rho=0.001,\;\;\gamma=0.9 \tag{49.92}
$$

Each iteration involves a run over $E=100$ episodes to train the critic. At the end of the simulation, we evaluate the policies at each state and arrive at the  matrix $\Pi$ in (49.93), with each row corresponding to one state and each column to one action  (i.e., each row corresponds to the distribution $\pi(a|s)$).  
The boxes highlight the largest entries in each column and are used to define the "optimal" policies at that state.  The resulting optimal actions are represented by arrows in Fig. 49.3. 
 
$$
\Pi=\begin{array}{ccccc|l}
\textnormal{ up}&\textnormal{ down}&\textnormal{ left}&\textnormal{ right}&\textnormal{ stop}&\textnormal{ state}\\\hline
0.0000  &  \fbox{0.9933}  &  0.0067  &  0.0000   &      0&\leftarrow s=1\\
\fbox{0.9825}  &  0.0008  &  0.0168  &  0.0000  &       0&\leftarrow s=2\\
\fbox{ 0.8348} &   0.0198 &   0.0341   & 0.1113 &        0&\leftarrow s=3\\
0    &     0    &     0   &      0    &     0&\leftarrow s=4\\
\fbox{0.9831} &   0.0013 &   0.0155 &   0.0001   &      0&\leftarrow s=5\\
0.0657  &  0.0013 &   \fbox{0.5922} &   0.3408   &      0&\leftarrow s=6\\
\fbox{0.9983}  &  0.0000 &   0.0017 &   0.0000  &       0&\leftarrow s=7\\
0      &   0      &   0      &   0  &  \fbox{1.0000}&\leftarrow s=8\\
\fbox{1.0000} &   0.0000 &   0.0000 &   0.0000 &        0&\leftarrow s=9\\
\fbox{0.9951} &   0.0000 &   0.0000 &   0.0049 &        0&\leftarrow s=10\\
0      &   0      &   0      &   0   &      0&\leftarrow s=11\\
\fbox{0.9974} &   0.0000 &   0.0013 &   0.0013  &       0&\leftarrow s=12\\
0.0032 &   0.0000 &   0.0001 &   \fbox{0.9967}  &       0&\leftarrow s=13\\
0.0001 &   0.0001 &   0.0000 &   \fbox{0.9998} &        0&\leftarrow s=14\\
0.0000 &   0.0000 &   0.0000 &   \fbox{1.0000} &        0&\leftarrow s=15\\
0      &   0      &   0      &   0   & \fbox{1.0000}&\leftarrow s=16\\
0      &   0      &   0      &   0   & \fbox{1.0000}&\leftarrow s=17
\end{array} \tag{49.93}
$$

In [21]:
# Natural gradient actor-critic algorithm

E = 100 # number of episodes within each iteration
max_episode_duration = 50
delta = 0
gamma = 0.9
mu = 0.01 # step-size for critic (w)
mug = 0.001 # step-size for actor (theta)
iter_ = 10000 # repeat for this many iterations to learn theta; each iteration has E episodes
use_reduced_features = 0 # set to zero to use the one-hot encoded extended feature

if use_reduced_features == 1:
    H = Hr 
    M = Mr 
    F = Fr 
    T = Tr 
else:
    H = He 
    M = Me 
    F = Fe 
    T = Te 

c = np.zeros(T) # linear value function model
theta = np.zeros(T) # parameter for Gibbs distribution

kernel = np.zeros(NS)

for m in tqdm(range(iter_)): # each iteration involves multiple episodes
    # Current Gibbs distribution using current model theta
    Pi_theta = compute_policy(F,theta,T,NA,NS,Pi)

    for e in range(E): # iterates over episodes
        counter = 0
        sample = 1
        while sample == 1:
            idx = np.random.randint(NS-1)+1 # select a random non-exit state index
            if (idx != 4) and (idx != 11) and (idx != 17): # excluding the block locations and exit state
                s = states[idx-1]
                sample = 0

    h = H[s, :].T # initial state feature vector
    policy = Pi_theta[:, s] # pi(a|s; theta) at this state s
    act = select_action(policy) # initial conditions
    idx = (s-1)*NA+act 
    f = F[int(idx), :].T # initial f vector

    while (s!=17) and (counter < max_episode_duration): # state s different from EXIT state
        for j in range(NS):
            kernel[j] = P[s, act, j]
        sprime = select_next_state(kernel) # next state
        hprime = H[sprime, :].T # next feature
        policyprime = Pi_theta[:, sprime] # pi(a|sprime; theta) at state prime
        actprime = select_action(policyprime)
        idx_prime = (sprime-1)*NA + actprime
        fprime = F[int(idx_prime), :].T 

        # CRITIC for state-action value function learning
        r = reward[s] # in this example, reward is only state s-dependent
        beta = r + (gamma*fprime - f).T@c
        c += mu*beta*f 
        s = sprime # set next sample
        act = actprime 
        f = fprime 
        counter += 1
    co = c.copy() # learned CRITIC MODEL

    #ACTOR
    theta += mug*co 

thetao = theta 
Pi_theta = compute_policy(F,thetao,T,NA,NS,Pi) # optimal policy
max_values = Pi_theta.max(axis=0)
indexes = np.argmax(Pi_theta, axis=0)

act = [0]*NS
action_states = [0]*NS
for s in range(NS):
    act[s] = indexes[s] # indexes of the permissible action  
    action_states[s] = actions[act[s]]
    print(s+1, action_states[s])

print("policy AxS from NATURAL actor critic algorithm")
print(Pi_theta)

100%|██████████| 10000/10000 [00:21<00:00, 457.27it/s]

1 up
2 up
3 right
4 stop
5 up
6 right
7 up
8 stop
9 up
10 up
11 stop
12 up
13 right
14 right
15 right
16 stop
17 stop
policy AxS from NATURAL actor critic algorithm
[[2.50000000e-01 9.96811025e-01 7.61399945e-02 0.00000000e+00
  9.99995654e-01 6.33866948e-04 9.99985042e-01 0.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  1.80143837e-15 4.34798524e-21 7.70863360e-26 0.00000000e+00
  0.00000000e+00]
 [2.50000000e-01 2.34082025e-04 5.84616955e-03 0.00000000e+00
  2.67094933e-07 1.02884545e-04 3.55168431e-14 0.00000000e+00
  3.68332691e-39 1.24565357e-26 0.00000000e+00 6.87469592e-14
  6.96842989e-18 1.97882261e-21 1.43232035e-30 0.00000000e+00
  0.00000000e+00]
 [2.50000000e-01 1.15305088e-04 4.13189217e-03 0.00000000e+00
  3.79221104e-06 1.03735388e-03 1.49584110e-05 0.00000000e+00
  4.21030226e-32 1.12328402e-21 0.00000000e+00 7.62091515e-12
  2.37084012e-18 1.04396673e-25 6.44393882e-33 0.00000000e+00
  0.00000000e+00]
 [2.50000000e-01 2.83958740e-03 9.138

## Example 49.8 (Playing a game over a grid)

We illustrate the operation of the PPO-clip method (49.163)   by reconsidering the same grid problem from  Fig. 48.1. We employ the same encoding for states $s$ and state--action pairs $(s,a)$ from Example 49.5.  For illustration purposes, we run the algorithm  for  1,000,000 iterations using constant step sizes throughout the simulation:

$$
\mu_1=0.01,\;\; \mu=0.0001, \;\;\rho=0.001,\;\;\gamma=0.9 \tag{49.164}
$$

Each iteration involves a run over $E=100$ episodes to train the critic. At the end of the simulation, we evaluate the policies at each state and arrive at the  matrix $\Pi$ in (49.165), with each row corresponding to one state and each column to one action  (i.e., each row corresponds to the distribution $\pi(a|s)$).  
The boxes highlight the largest entries in each column and are used to define the "optimal" policies at that state.  The resulting optimal actions are represented by arrows in Fig. 49.4. 
 
$$
\Pi=\begin{array}{ccccc|l}
\textnormal{ up}&\textnormal{ down}&\textnormal{ left}&\textnormal{ right}&\textnormal{ stop}&\textnormal{ state}\\\hline
0.0469  &  0.2984 &   \fbox{0.5393} &   0.1154 &        0&\leftarrow s=1\\
\fbox{0.4030} &   0.1399 &   0.3889 &   0.0681 &        0&\leftarrow s=2\\
\fbox{0.4543} &   0.1747 &   0.2237 &   0.1472 &        0&\leftarrow s=3\\
0      &   0      &   0      &   0      &   0&\leftarrow s=4\\
\fbox{0.6573} &   0.1065 &   0.1546  &  0.0817 &        0&\leftarrow s=5\\
0.2107 &   0.1208 &   \fbox{0.4925} &   0.1760 &        0&\leftarrow s=6\\
\fbox{0.8450} &   0.0313 &   0.1090  &  0.0147 &        0&\leftarrow s=7\\
0      &   0      &   0       &  0 &   \fbox{1.0000}&\leftarrow s=8\\
\fbox{0.9136} &   0.0137 &   0.0352  &  0.0375 &        0&\leftarrow s=9\\
\fbox{0.8824} &   0.0196 &   0.0440  &  0.0541 &        0&\leftarrow s=10\\
0      &   0      &   0       &  0      &   0&\leftarrow s=11\\
\fbox{0.8148} &   0.0393 &   0.0730  &  0.0729 &        0&\leftarrow s=12\\
0.0823 &   0.0417 &   0.0545 &   \fbox{0.8215} &        0&\leftarrow s=13\\
0.0445 &   0.0445 &   0.0238  &  \fbox{0.8872} &        0&\leftarrow s=14\\
0.0278 &   0.0144 &   0.0134  &  \fbox{0.9445} &        0&\leftarrow s=15\\
0      &   0      &   0      &   0   & \fbox{1.0000}&\leftarrow s=16\\
0      &   0      &   0      &   0  &  \fbox{1.0000}&\leftarrow s=17
\end{array} \tag{49.165}
$$

In [9]:
# PPO Clipped Cost Method for optimal policy design

E = 100 # number of episodes within each iteration
max_episode_duration = 50
epsilon = 0.02 # bound of KL divergence
ar = np.random.randint(4)+1 # arbitrary reference action from {UP, DOWN, LEFT, RIGHT}
delta = 0
gamma = 0.9
mu = 0.01 # step-size for critic (w)
mug = 0.0001 # step-size for actor (theta)
iter_ = 10000 # repeat this many iterations to learn theta; each iteration has E episodes
use_reduced_features = 0 # set to zero to use the one-hot encoded extended feature

if use_reduced_features == 1:
    H = Hr 
    M = Mr 
    F = Fr 
    T = Tr 
else:
    H = He 
    M = Me 
    F = Fe 
    T = Te 

w = np.zeros(M) # linear value funciton model
theta = np.zeros(T) # parameter for Gibbs distribution

saved_states = [] # neet to save states, features, rewards, actions during all episodes
saved_features = []
saved_rewards = []
saved_actions = []

kernel = np.zeros(NS)

for m in tqdm(range(iter_)): # each iterations involves multiple episodes
    # Current Gibbs distribution using current model theta
    Pi_theta = compute_policy(F, theta, T, NA, NS, Pi)
    counter_transitions = 0

    for e in range(E): # iterates over episodes
        counter = 0
        sample = 1 # select initial sample of episodes to feed into network

        while sample == 1:
            idx = np.random.randint(NS-1)+1 # select a random non-exit state index
            if (idx != 4) and (idx != 11) and (idx != 17): # excluding the block locations and exit state
                s = states[idx-1]
                sample = 0

        while (s!=17) and (counter < max_episode_duration): # state s different from EXIT
            h = H[s, :].T # state feature vector
            policy = Pi_theta[:, s] # pi(a|s;theta) at state s
            act = select_action(policy)

            for j in range(NS):
                kernel[j] = P[s, act, j]
            
            sprime = select_next_state(kernel) # next state
            hprime = H[sprime, :].T # next feature

            # CRITIC for state value function learning
            r = reward[s] # in this example reward is only state s-dependent
            delta = r + (gamma*hprime - h).T@w 
            w += mu*delta*h 

            # save visited states, actions, rewards, internal signals within neural network
            saved_features.append([h, hprime])
            saved_states.append([s, sprime])
            saved_rewards.append(r)
            saved_actions.append(act)

            s = sprime # set next sample
            counter += 1
            counter_transitions += 1
    wo = w

    # ACTOR
    N = counter_transitions - 1
    g_vec = np.zeros(T) # g_{m-1}
    z_vec = np.zeros(T) # z_{m-1}

    # determining the gradient vectors g_{m-1} and z_{m-1}
    for nn in range(N):
        hx = saved_features[nn][0]
        hxprime = saved_features[nn][1]
        sx = saved_states[nn][0]
        sxprime = saved_states[nn][1]
        rx = saved_rewards[nn]
        ax = saved_actions[nn]

        # determing g_{m-1}
        deltax = rx + (gamma*hxprime - hx).T@wo
        x_vec = compute_gradient_log_pi(F,Pi_theta,sx,ax,NA) # gradient of log pi =(f-f_bar)= grad ell
        g_vec += deltax*x_vec 

        # determining z_{m-1}
        Ax = deltax # estimated advantage value
        if Ax >= 0:
            cx = (1+epsilon)*Ax 
        else:
            cx = (1-epsilon)*Ax 
        
        diffx = Ax - cx # since ell(theta) = A
        if diffx >= 0:
            signx = +1
        else:
            signx = -1
        
        y_vec = .5*x_vec*Ax*(1-signx) # x_vec is f-f_bar
        z_vec += y_vec 
    
    g_vec = g_vec/N 
    z_vec = z_vec/N 
    theta_hat = theta + mug*z_vec 

    KL_div = 0

    # determining average KL divergence
    for nn in range(N):
        sx = saved_states[nn][0]
        sum_y = 0
        sum_yprime = 0
        sum_expyprime = 0
        sum_expy = 0

        idx_sa_r = (sx-1)*NA + ar 
        f_sa_r = F[int(idx_sa_r), :].T # ar is reference action

        y = np.zeros(NA)
        yprime = np.zeros(NA)
        expy = np.zeros(NA)
        expyprime = np.zeros(NA)
        expy2 = np.zeros(NA)
        for a in range(NA):
            idx_sa = (sx-1)*NA + a 
            f_sa = F[int(idx_sa), :]
            y[a] = (f_sa - f_sa_r).T@theta 
            yprime[a] = (f_sa - f_sa_r).T@theta_hat
            expy[a] = np.exp(y[a])
            expyprime[a] = np.exp(yprime[a])
            expy2[a] = (np.exp(y[a]))*(y[a]-yprime[a])

        sum_expy = expy.sum()
        sum_expyprime = expyprime.sum()
        sum_expy2 = expy2.sum()

        KL_div += (np.log(sum_expyprime/sum_expy)) + (sum_expy2/sum_expy)
    KL_div = KL_div / N 

    if (g_vec.T@theta_hat >= 0) and (KL_div <= epsilon):
        theta = theta_hat

thetao = theta 
Pi_theta = compute_policy(F,thetao,T,NA,NS,Pi) # optimal policy
max_values = Pi_theta.max(axis=0)
indexes = np.argmax(Pi_theta, axis=0)

act = [0]*NS
action_states = [0]*NS
for s in range(NS):
    act[s] = indexes[s] # indexes of the permissible action  
    action_states[s] = actions[act[s]]
    print(s+1, action_states[s])

print("policy AxS from PPO Clipped Cost algorithm")
print(Pi_theta)

100%|██████████| 10000/10000 [1:51:49<00:00,  1.49it/s]  

1 up
2 left
3 left
4 stop
5 up
6 left
7 left
8 stop
9 up
10 up
11 stop
12 up
13 right
14 right
15 right
16 stop
17 stop
policy AxS from PPO Clipped Cost algorithm
[[0.25       0.25007042 0.25017397 0.         0.25035637 0.25017277
  0.25117583 0.         0.25201217 0.25124575 0.         0.25064599
  0.25025333 0.24981959 0.25073408 0.         0.        ]
 [0.25       0.25011921 0.25010356 0.         0.25004421 0.25025705
  0.25044727 0.         0.24846733 0.24879504 0.         0.2495117
  0.24967574 0.24968928 0.24865468 0.         0.        ]
 [0.25       0.25110673 0.25032682 0.         0.25028457 0.25080987
  0.25207201 0.         0.24929606 0.25000439 0.         0.24986916
  0.24973089 0.24927058 0.24920315 0.         0.        ]
 [0.25       0.24870364 0.24939565 0.         0.24931485 0.2487603
  0.24630489 0.         0.25022444 0.24995482 0.         0.24997315
  0.25034004 0.25122055 0.25140809 0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0

## Example 49.9 (Playing a game over a grid)

We illustrate the operation of the deep learning A2C implementation (49.179)  by reconsidering the same grid problem from  Fig. 48.1. We consider a feedforward neural network with $L=4$ layers (including one input layer, one output layer, and two hidden layers). All neurons employ the ReLU activation function except for the output layer, which employs a softmax mapping. The step-size, regularization, and discount factor parameters are set to:

$$
\mu_1=0.01,\;\;\;\mu=0.00001, \;\;\rho=0.001,\;\;\gamma=0.9 \tag{49.180}
$$

We employ the same one-hot encoded feature vectors  (49.80). We run the deep learning algorithm for 500,000 iterations to learn the parameters $\{W_{\ell},\theta_{\ell}\}$. Each iteration involves a run over $E=100$ episodes to train the critic. At the end of the simulation, we evaluate the policies at each state (we feed feature vector $h_s$ and determine $\widehat{\gamma}_s=\pi^{\star}(a|s)$). We arrive at the  matrix $\Pi$ in (49.181), with each row corresponding to one state and each column to one action (i.e., each row corresponds to the distribution $\pi^{\star}(a|s)$). 
The boxes highlight the largest entries in each column and are used to define the "optimal" policies at that state.  The resulting optimal actions are represented by arrows in Fig. 49.6.
 
$$
\Pi=\begin{array}{ccccc|l}
\textnormal{ up}&\textnormal{ down}&\textnormal{ left}&\textnormal{ right}&\textnormal{ stop}&\textnormal{ state}\\\hline
0.0077  &  0.0014 &   \fbox{0.9619} &   0.0290 &        0&\leftarrow s=1\\
\fbox{0.9658} &   0.0001 &   0.0342 &   0.000 &        0&\leftarrow s=2\\
\fbox{0.9792} &   0.000 &   0.0208 &   0.0000 &        0&\leftarrow s=3\\
0      &   0      &   0      &   0      &   0&\leftarrow s=4\\
\fbox{0.9911} &   0.000 &   0.0089  &  0.000 &        0&\leftarrow s=5\\
0.0178 &   0.0016 &   \fbox{0.9631} &   0.0175 &        0&\leftarrow s=6\\
\fbox{0.9855} &   0.000 &   0.0145  &  0.000 &        0&\leftarrow s=7\\
0      &   0      &   0       &  0 &   \fbox{1.0000}&\leftarrow s=8\\
\fbox{0.9980} &   0.000 &   0.0020  &  0.000 &        0&\leftarrow s=9\\
\fbox{0.9975} &   0.000 &   0.0025  &  0.000 &        0&\leftarrow s=10\\
0      &   0      &   0       &  0      &   0&\leftarrow s=11\\
\fbox{0.9960} &   0.000 &   0.0040  &  0.000 &        0&\leftarrow s=12\\
0.000 &   0.000 &   0.0043 &   \fbox{0.9957} &        0&\leftarrow s=13\\
0.000 &   0.000 &   0.0019  &  \fbox{0.9981} &        0&\leftarrow s=14\\
0.000 &   0.000 &   0.0012  &  \fbox{0.9988} &        0&\leftarrow s=15\\
0      &   0      &   0      &   0   & \fbox{1.0000}&\leftarrow s=16\\
0      &   0      &   0      &   0  &  \fbox{1.0000}&\leftarrow s=17
\end{array} \tag{49.181}
$$

In [6]:
# Deep advantage actor-critic (A2C) algorithm for optimal policy design

E = 100 # number of episodes within each iteration
max_episode_duration = 50
delta = 0
gamma = 0.9
epsilon = 0.1 # for epsilon-greedy exploration
mu = 0.01 # step-size for critic (w)
mug = 0.00001 # step-size for actor (theta)
rho = 0.001 # regularization parameter
iter_ = 10000 # repeat for this many iterations to learn theta; each iteration has E episodes
use_reduced_features = 0 # set to zero to use one-hot encoded etended feature

if use_reduced_features == 1:
    H = Hr 
    M = Mr 
    F = Fr 
    T = Tr 
else:
    H = He 
    M = Me 
    F = Fe 
    T = Te 

w = np.zeros(M) # critic model

# parameters of neural network
L = 4 # number of neural network layers
n1 = M # number of neurons in input layer = size of feature vectors without bias entry
n2 = 32 # number of neurons in first hidden layer
n3 = 32 # number of neurons in second hidden layer
n4 = NA # number of neurons in output layer = number of actions available
type_activation = [-1, 3, 3, 4] # -1 irrelevant (not used), 0: linear, 1: sigmoid, 2: tanh, 3: rectifier; 4: softmax

W1 = (1/np.sqrt(n1))*np.random.randn(n2, n1) # weight matrices
W2 = (1/np.sqrt(n2))*np.random.randn(n3, n2) 
W3 = (1/np.sqrt(n3))*np.random.randn(n4, n3)
Wcell = [W1, W2, W3] # a cell array containing the weight matrices of different dimensions 
Wcell_before = Wcell.copy()

theta1 = np.random.randn(n2) # bias vectors; NOT USED in this simulation
theta2 = np.random.randn(n3)
theta3 = np.random.randn(n4)
ThetaCell = [theta1, theta2, theta3] # a cell array for the thetas
ThetaCell_before = ThetaCell.copy()

use_theta = 1 # need to use theta when output is softmax, otherwise h = 0 maps into uniform distribution for state 17

saved_states = [] # neet to save states, features, rewards, actions during all episodes
saved_features = []
saved_rewards = []
saved_actions = []
saved_y_values = []
saved_z_values = []

kernel = np.zeros(NS)

for m in tqdm(range(iter_)): # each iteration involves multiple episodes 
    counter_transitions = 1 # counts # of state transitions 
    
    for e in range(E): # iterates over episodes
        counter = 0
        sample = 1 # select initial sample of episodes to feed into network

        while sample == 1:
            idx = np.random.randint(NS-1)+1 # select a random non-exit state index
            if (idx != 4) and (idx != 11) and (idx != 17): # excluding the block locations and exit state
                s = states[idx-1]
                sample = 0

        while (s!=17) and (counter < max_episode_duration): # state s different from EXIT state
            h = H[s, :].T # state feature vector
            policy = Pi[:, s] # true policy --> helps identify permissible actions at state s
            flag = policy > 0
            yCellf, zCellf = feed_forward_permissible(Wcell,ThetaCell,L,type_activation,h,use_theta,flag)
            policy_hat = yCellf[-1]
            act = select_action(policy_hat)

            for j in range(NS):
                kernel[j] = P[s, act, j]
            
            sprime = select_next_state(kernel) # next state
            hprime = H[sprime, :].T # next feature

            # CRITIC for state value function learning
            r = reward[s] # in this example reward is only state s-dependent
            delta = r + (gamma*hprime - h).T@w 
            w += mu*delta*h 

            # save visited states, actions, rewards, internal signals within neural network
            saved_features.append([h, hprime])
            saved_states.append([s, sprime])
            saved_rewards.append(r)
            saved_actions.append(act)
            saved_y_values.append(yCellf)
            saved_z_values.append(zCellf)

            s = sprime # set next sample
            counter += 1
            counter_transitions += 1 # plays role of N

    wo = w.copy() # learned CRITIC model

    # ACTOR
    N = counter_transitions - 1
    sum_g = np.zeros(T)
    saved_sigma = [[np.zeros(1)]*L]*N 

    for nn in range(N):
        hx = saved_features[nn][0]
        hxprime = saved_features[nn][1]
        sx = saved_states[nn][0]
        sxprime = saved_states[nn][1]
        rx = saved_rewards[nn]
        ax = saved_actions[nn]

        yCellf = saved_y_values[nn]
        gamma_hat = yCellf[-1]
        ea = np.zeros(NA)
        ea[ax] = 1

        deltax = rx + (gamma*hxprime - hx).T@wo 
        saved_sigma[nn][-1] = deltax*(ea-gamma_hat)

    Wcellf, ThetaCellf = feed_back_batch(Wcell,ThetaCell,saved_y_values,saved_z_values,saved_sigma,L,type_activation,mu,rho,use_theta,N)
    Wcell = Wcellf.copy()
    ThetaCell = ThetaCellf.copy()



In [5]:
Wstar = Wcell.copy()
Thetastar = ThetaCell.copy()

Pi_hat =np.zeros((NA, NS))
act_vec = np.zeros(NA)


action_states = [None]*NS
for s in range(NS):
    if (s!=3) and (s!=10): # not valid states
        h = H[s, :].T # feature vector
        policy = Pi[:, s] # true policy --> helps identify permissible actions at state s
        flag = policy > 0
        yCell, zCell = feed_forward_permissible(Wstar,Thetastar,L,type_activation,h,use_theta,flag)
        policy_hat = yCell[-1]
        act = select_action(policy_hat)
        action_states[s] = actions[act]
        print(s+1, action_states[s])
        Pi_hat[:, s] = policy_hat

print(Pi_hat.T)

1 up
2 left
3 up
5 up
6 up
7 down
8 stop
9 right
10 down
12 down
13 left
14 left
15 down
16 stop
17 stop
[[0.25952025 0.28297748 0.22339837 0.23410391 0.        ]
 [0.25934858 0.26722509 0.18410842 0.28931791 0.        ]
 [0.23027523 0.27020438 0.22197813 0.27754226 0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.25575507 0.24816901 0.22949104 0.26658488 0.        ]
 [0.25339495 0.27534248 0.23662773 0.23463484 0.        ]
 [0.24113869 0.25493585 0.24435725 0.25956821 0.        ]
 [0.         0.         0.         0.         1.        ]
 [0.24406735 0.27513779 0.24466741 0.23612744 0.        ]
 [0.25136462 0.26438274 0.23200873 0.2522439  0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.24559798 0.26951402 0.22689835 0.25798965 0.        ]
 [0.23590059 0.28488897 0.23828841 0.24092203 0.        ]
 [0.24972788 0.2697781  0.22243727 0.25805675 0.        ]
 [0.23100449 0.27117174 0.20172471 0.29609906 0.        ]
 [0.         0.         0